In [9]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import itertools

In [10]:
pathDBMS = "/Users/lieshaouas/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-542ff0e0-d83e-4c83-af3b-c0876324d6e2/import/"
fileName_donation = "Donations"
fileName_donor = "Donors"
fileName_project = "Projects"

In [5]:
df_donation = pd.read_csv(fileName_donation+'.csv');

In [6]:
df_donor = pd.read_csv(fileName_donor+".csv");

In [7]:
df_project = pd.read_csv(fileName_project+".csv");

In [7]:
print("df_donation",df_donation.shape)
print("df_donor",df_donor.shape)
print("df_project",df_project.shape)

df_donation (4687884, 7)
df_donor (2122640, 5)
df_project (1110017, 18)


In [8]:
list_df = [df_donation,df_donor,df_project]

In [106]:
def cleanDf(df):
    df.columns = [c.replace(" ","_") for c in df.columns]
    for column in df.columns:
        if df[column].isnull().sum() > 0:   
            df.drop(df[df[column].isnull()].index, inplace=True)

In [10]:
for df in list_df:
    cleanDf(df)

In [118]:
def deleteNotIn(df1,df2,columnid):
    test = set(df1[columnid]) & set(df2[columnid])
    indexNames1 = df1[df1[columnid].apply(lambda x: False if x in test else True)].index
    indexNames2 = df2[df2[columnid].apply(lambda x: False if x in test else True)].index
    df1.drop(indexNames1 , inplace=True)
    df2.drop(indexNames2 , inplace=True)

In [119]:
deleteNotIn(df_school,df_project,"School_ID")

In [122]:
deleteNotIn(df_teacher,df_project,"Teacher_ID")

In [23]:
df_project = df_project[['Project_ID', 'School_ID', 'Teacher_ID',
       'Teacher_Project_Posted_Sequence', 'Project_Type', 'Project_Title',
       'Project_Subject_Category_Tree', 'Project_Subject_Subcategory_Tree',
       'Project_Grade_Level_Category', 'Project_Resource_Category',
       'Project_Cost', 'Project_Posted_Date', 'Project_Expiration_Date',
       'Project_Current_Status', 'Project_Fully_Funded_Date']]

In [43]:
def makeNewColumnWithDate(df,columnDateName):
    df[columnDateName] = df[columnDateName].str.replace('-','/')
    df["year"] = pd.to_datetime(df[columnDateName],format='%Y/%m/%d').dt.year
    #Création d'une nouvelle colonne day_of_week, qui permet d'avoir le jour de la semaine lundi ou mardi etc...
    df['day_of_week'] = pd.to_datetime(df[columnDateName],format='%Y/%m/%d %H:%M').dt.dayofweek
    df["isWeekend"] =  np.where(df["day_of_week"] < 5, 1, 0)
    df["isMorning"] =  np.where((df["hour"] <= 12), 1, 0)
    df['day_of_week'] = pd.to_datetime(df[columnDateName],format='%Y/%m/%d %H:%M').dt.dayofweek
    df[columnDateName] = df[columnDateName].str.replace('/','-')
    df[columnDateName] = df[columnDateName]''.str[:10]

In [44]:
makeNewColumnWithDate(df_donation,"Donation_Received_Date")

In [46]:
df_donation.head()

,Project_ID,Donation_ID,Donor_ID,Donation_Included_Optional_Donation,Donation_Amount,Donor_Cart_Sequence,Donation_Received_Date,year,month,day,hour,min,day_of_week,isWeekend,isMorning
0,000009891526c0ade7180f8423792063,688729120858666221208529ee3fc18e,1f4b5b6e68445c6c4a0509b3aca93f38,No,178.37,11,2016-08-23,2016,8,23,0,0,1,1,1
1,000009891526c0ade7180f8423792063,dcf1071da3aa3561f91ac689d1f73dee,4aaab6d244bf3599682239ed5591af8a,Yes,25.00,2,2016-06-06,2016,6,6,0,0,0,1,1
2,000009891526c0ade7180f8423792063,18a234b9d1e538c431761d521ea7799d,0b0765dc9c759adc48a07688ba25e94e,Yes,20.00,3,2016-06-06,2016,6,6,0,0,0,1,1
3,000009891526c0ade7180f8423792063,38d2744bf9138b0b57ed581c76c0e2da,377944ad61f72d800b25ec1862aec363,Yes,25.00,1,2016-05-15,2016,5,15,0,0,6,0,1
4,000009891526c0ade7180f8423792063,5a032791e31167a70206bfb86fb60035,6d5b22d39e68c656071a842732c63a0c,Yes,25.00,2,2016-05-17,2016,5,17,0,0,1,1,1


In [29]:
result = pd.merge(df_donation, df_project, on=["Project_ID"])

In [30]:
result = pd.merge(result, df_donor, on=["Donor_ID"])

In [35]:
df_donation_sample = result[['Project_ID', 'Donation_ID', 'Donor_ID',
       'Donation_Included_Optional_Donation', 'Donation_Amount',
       'Donor_Cart_Sequence', 'Donation_Received_Date']]

In [36]:
df_project_sample = result[['Project_ID', 'School_ID', 'Teacher_ID',
       'Teacher_Project_Posted_Sequence', 'Project_Type', 'Project_Title',
       'Project_Essay', 'Project_Short_Description', 'Project_Need_Statement',
       'Project_Subject_Category_Tree', 'Project_Subject_Subcategory_Tree',
       'Project_Grade_Level_Category', 'Project_Resource_Category',
       'Project_Cost', 'Project_Posted_Date', 'Project_Expiration_Date',
       'Project_Current_Status', 'Project_Fully_Funded_Date']]

In [37]:
df_donor_sample = result[['Donor_ID', 'Donor_City', 'Donor_State', 'Donor_Is_Teacher',
       'Donor_Zip']]

In [47]:
df_donation_sample_copy = df_donation[:1000000]

In [51]:
print("df_donation",df_donation_sample_copy.shape)
print("df_donor",df_donor_sample_copy.shape)
print("df_project",df_project_sample_copy.shape)

df_donation (1000000, 15)
df_donor (1000000, 5)
df_project (710666, 15)


In [58]:
print("df_donation",df_donation_sample_copy.shape)
print("df_donor",df_donor_sample_copy.shape)
print("df_project",df_project_sample_copy.shape)

df_donation (604626, 15)
df_donor (334865, 5)
df_project (184284, 15)


In [52]:
df_donation_sample_copy.drop_duplicates(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
df_donor_sample_copy.drop_duplicates(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [123]:
df_project.drop_duplicates(inplace=True)

In [56]:
deleteNotIn(df_project_sample_copy,df_donation_sample_copy,'Project_ID')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [57]:
deleteNotIn(df_donor_sample_copy,df_donation_sample_copy,'Donor_ID')

In [126]:
df_project = df_project[['Project_ID', 'School_ID', 'Teacher_ID',
       'Teacher_Project_Posted_Sequence', 'Project_Type', 'Project_Title',
       'Project_Subject_Category_Tree', 'Project_Subject_Subcategory_Tree',
       'Project_Grade_Level_Category', 'Project_Resource_Category',
       'Project_Cost', 'Project_Posted_Date', 'Project_Expiration_Date',
       'Project_Current_Status', 'Project_Fully_Funded_Date']]

In [132]:
df_donation_sample_copy.to_csv(pathDBMS+fileName_donation+'A.csv',index=False) 

In [133]:
df_donor_sample_copy.to_csv(pathDBMS+fileName_donor+'A.csv',index=False) 

In [134]:
df_project_sample_copy.to_csv(pathDBMS+fileName_project+'A.csv',index=False) 